# Create Submission File

## Imports and get data

In [2]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import numpy as np
from HelpFunctions.calc_score import evaluate_horizon
from HelpFunctions.mix_models import mix_models
from HelpFunctions.date_and_time import most_recent_thursday, split_time

from DAX.HelpFunctions.get_dax_data import get_dax_data
from DAX.Models import baseline as dax_bl
from DAX.Models import garch11 as dax_garch

from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
from Energy.Models import baseline as en_bl
from Energy.Models import Model4_population as en_m4_pop
from datetime import datetime
import pytz

# import importlib
# importlib.reload(Energy.Models)

# import importlib
# importlib.reload(Energy.Models)

/Users/elias/Desktop/PTSFC/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### DAX: Get and prepare Data
Use tuesday as last datapoint

In [4]:
# Insert Date of Wednesday (forecast-day) here. The current day has to be wednesday!!
start_date = datetime(2023, 12, 13, 8, 0, 0)
assert start_date.weekday() == 2

In [5]:
df_dax = get_dax_data()
# pretend_start_date = date(2023, 11, 22, 11, 00, 0, tzinfo=pytz.timezone('Europe/Berlin'))

df_dax = df_dax[df_dax.index.date <= start_date.date()]

In [6]:
df_dax

,Close,ret1,ret2,ret3,ret4,ret5
Date,,,,,,
1987-12-30 00:00:00+01:00,1005.190002,NaN,NaN,NaN,NaN,NaN
1988-01-04 00:00:00+01:00,956.489990,-4.966154,NaN,NaN,NaN,NaN
1988-01-05 00:00:00+01:00,996.099976,4.057731,-0.908423,NaN,NaN,NaN
1988-01-06 00:00:00+01:00,1006.010010,0.989967,5.047698,0.081544,NaN,NaN
1988-01-07 00:00:00+01:00,1014.469971,0.837426,1.827393,5.885123,0.918970,NaN
...,...,...,...,...,...,...
2023-12-06 00:00:00+01:00,16656.439453,0.743190,1.522537,1.566682,2.683366,2.985878
2023-12-07 00:00:00+01:00,16628.990234,-0.164932,0.578257,1.357605,1.401750,2.518433
2023-12-08 00:00:00+01:00,16759.220703,0.780102,0.615170,1.358360,2.137707,2.181852


### Energy: Get and prepare Data

In [7]:
from Energy.HelpFunctions.get_energy_data import fetch_energy_data
fetch_energy_data()

  0%|          | 0/260 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 260/260 [00:22<00:00, 11.42it/s]


In [3]:
df_en = get_energy_data(force_return=True)
df_en = prepare_data(df_en)

/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:17: UserWarning: The data is not up to date anymore. Please call fetch_energy_data
  warnings.warn("The data is not up to date anymore. Please call fetch_energy_data", UserWarning)


In [4]:
df_en = df_en.loc[df_en.index <= start_date]

NameError: name 'start_date' is not defined

In [5]:
get_energy_data(force_return=True)

/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:17: UserWarning: The data is not up to date anymore. Please call fetch_energy_data
  warnings.warn("The data is not up to date anymore. Please call fetch_energy_data", UserWarning)


,Netzlast_Gesamt
date_time,
2018-12-24 00:00:00,42029.25
2018-12-24 01:00:00,39610.25
2018-12-24 02:00:00,39138.75
2018-12-24 03:00:00,39421.00
2018-12-24 04:00:00,40747.75
...,...
2023-12-04 15:00:00,70153.50
2023-12-04 16:00:00,70703.50
2023-12-04 17:00:00,72672.00


## Run models for the data

### DAX

In [14]:
from DAX.Models.arma_garch_11 import arma_garch_11
from DAX.Models.baseline import baseline
# dax_bl.baseline(df_dax)
# forecast_dax = dax_garch.garch11(df_dax)
forecast_dax = mix_models([arma_garch_11, baseline], [1,1], df_dax, 'DAX')
forecast_dax

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-13,DAX,1 day,-3.251335,-0.632117,0.064390,0.781185,3.008683
1,2023-12-14,DAX,2 day,-3.636778,-0.721054,0.136271,0.992888,3.554754
2,2023-12-15,DAX,5 day,-4.679263,-0.807332,0.269508,1.346280,4.724116
3,2023-12-18,DAX,6 day,-5.501670,-0.952888,0.279832,1.510589,5.385541
4,2023-12-19,DAX,7 day,-5.631634,-0.858713,0.473039,1.727851,5.994030


### Energy

In [9]:
from Energy.Models.baseline import baseline
from HelpFunctions.mix_models import mix_models_per_horizon
from Energy.Models.Model4_sunhours import model4_sunhours
from Energy.Models.Model4_holidays_2 import model4_holidays_2
from Energy.Models.Model4_population import model4_population
from Energy.Models.Model5 import model5

# en_bl.baseline(df_en, LAST_IDX=-1, offset_horizons=11)

weights_m5_bl_m4_x = [1,1,1,1,1]
functions_m5_bl_m4_x = [model5, baseline, model4_population, model4_holidays_2, model4_sunhours]




# forecast_en = en_m4_pop.model4_population(df_en, offset_horizons=15)
forecast_en = mix_models(functions_m5_bl_m4_x, weights_m5_bl_m4_x, df_en, target="energy", offset_horizons=52)
forecast_en

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-08 12:00:00,energy,36 hour,51.211792,60.978822,66.006145,69.591613,75.928808
1,2023-12-08 16:00:00,energy,40 hour,50.441191,58.550079,63.146476,67.019122,73.616147
2,2023-12-08 20:00:00,energy,44 hour,48.175862,54.811355,58.872356,62.406643,69.007995
3,2023-12-09 12:00:00,energy,60 hour,43.078250,51.933858,56.258513,59.377057,64.638557
4,2023-12-09 16:00:00,energy,64 hour,42.099342,49.725203,53.667444,56.957228,62.648578
5,2023-12-09 20:00:00,energy,68 hour,39.933314,46.470891,49.677924,52.847549,58.674600


### Infections

In [ ]:
forecast_infections = {
    'forecast_date': [start_date.strftime('%Y-%m-%d')] * 5,
    'target': ['infections'] * 5,
    'horizon': ['0 week', '1 week', '2 week', '3 week', '4 week'],
    # 'q0.025': ['NA'] * 5,
    # 'q0.25': ['NA'] * 5,
    # 'q0.5': ['NA'] * 5,
    # 'q0.75': ['NA'] * 5,
    # 'q0.975': ['NA'] * 5,
    'q0.025': [np.nan] * 5,
    'q0.25': [np.nan] * 5,
    'q0.5': [np.nan] * 5,
    'q0.75': [np.nan] * 5,
    'q0.975': [np.nan] * 5
}

forecast_infections = pd.DataFrame(forecast_infections)

In [ ]:
forecast_infections

## Merge results

Check results: Are they valid, forecast_date is right? The last entry might be cut off

In [ ]:
forecasts = pd.concat([forecast_dax,forecast_en, forecast_infections]).reset_index().drop(columns=['index'])
forecasts

If everything seems good: Override forecast_date

In [ ]:
forecasts['forecast_date'] = [start_date.strftime('%Y-%m-%d') for _ in range(len(forecasts))]

In [ ]:
forecasts

### Check df & Save csv-File

In [ ]:
from HelpFunctions.check_submission import check_submission_df

check_submission_df(forecasts)
forecasts.to_csv(f'./Abgaben/{start_date.strftime("%Y%m%d")}_JakePeralta.csv', na_rep='NA', index=False)